In [1]:
cd ../

/Users/rsmith/PycharmProjects/blackjack_python


In [2]:
import numpy as np
import pandas as pd
import time
import json
import pickle
import mdptoolbox

from enum import Enum
from scipy.sparse import csr_matrix

from src.simulator.deck import CARD_VALUES
from src.simulator.player import PlayerAction
from src.utils import tic, toc

## Basic Usage of MDP Solver

Assumes a model with no knowledge of card count

In [3]:
class TerminalState(Enum):
    Won = 0
    Lost = 1
    Pushed = 2
    Won2 = 3
    Lost2 = 4
    BJ = 5
    FINAL_STATE = 6
    BAD_OP = 7


# Have these states for after a split to still give reward on the hand
# TODO: Account for dealer card here as well as split_on
class SplitStateEnum(Enum):
    SWon = 0
    SLost = 1
    SPushed = 2
    SBJ = 3
    SWon2 = 4
    SLost2 = 5

NUM_INTERMEDIATE_STATES = 6


# Assuming drawing from deck with replacement
NEXT_CARD_PROBS = {
    card_val: (4.0 / 13 if card_val == 10 else 1.0 / 13)
    for card_val in range(1, 11)
}
NUM_CARDS = len(NEXT_CARD_PROBS)

# Getting conditional probabilities for the score a dealer gets based on
# their opening card
with open('data/tmp/dealer_result_conditional_probs.pkl', 'rb') as tmp_file:
    dealer_score_cond_probs = pickle.load(tmp_file)

DEALER_SCORE_MAP = {
    key: {}
    for key, _ in dealer_score_cond_probs
}
for (dealer_start, dealer_end), prob in dealer_score_cond_probs.items():
    # Setting busts to 0 so our comparison is fast
    dealer_end = dealer_end if dealer_end < 22 else 0
    
    DEALER_SCORE_MAP[dealer_start][dealer_end] = prob


In [4]:
# Defining actions and states

actions = [
    PlayerAction.STAY,
    PlayerAction.HIT,
    PlayerAction.SPLIT,
    PlayerAction.DOUBLE
]

# Define terminal states, that transition to a final FINAL_STATE
# where the reward is applied
terminal_states = [
    TerminalState.Won,
    TerminalState.Lost,
    TerminalState.Pushed,
    TerminalState.Won2,
    TerminalState.Lost2,
    TerminalState.BJ,
    TerminalState.FINAL_STATE,
    TerminalState.BAD_OP
]

# Have a split terminal state of each kind for:
# each card to split AND each possible dealer hand
split_intermediate_states = [
    SplitStateEnum.SWon,
    SplitStateEnum.SLost,
    SplitStateEnum.SPushed,
    SplitStateEnum.SBJ,
    SplitStateEnum.SWon2,
    SplitStateEnum.SLost2,
] * NUM_CARDS * NUM_CARDS

static_states = terminal_states + split_intermediate_states

hand_states = [
    ndx for ndx in range(4, 22)
]

dealer_card_states = [
    ndx for ndx in range(1, 11)
]

is_soft_states = [0, 1]
can_double_states = [0, 1]
can_split_states = [0, 1]

# Going to use 0 to represent no split here
split_on_states = [
    ndx for ndx in range(11)
]

non_terminal_states = [
    hand_states, dealer_card_states,
    is_soft_states, can_double_states, can_split_states,
    split_on_states
]

states_len = len(static_states) + np.prod([
    len(states) for states in non_terminal_states
])

In [5]:
def terminal_transitions(state):
    """
    Returns a transition vector for the terminal state given
    :return: ndarray of shape (len(states),)
    """
    vec = np.zeros(states_len)
    vec[TerminalState.FINAL_STATE.value] = 1
    return vec


def _reward_amount(state):
    """
    Helper that returns the reward amount
    """
    if state in [TerminalState.Won, SplitStateEnum.SWon]:
        return 1
    elif state in [TerminalState.Won2, SplitStateEnum.SWon2]:
        return 2
    elif state in [TerminalState.Lost, SplitStateEnum.SLost]:
        return -1
    elif state in [TerminalState.Lost2, SplitStateEnum.SLost2]:
        return -2
    elif state in [TerminalState.Pushed, SplitStateEnum.SPushed]:
        return 0
    elif state in [TerminalState.BJ, SplitStateEnum.SBJ]:
        return 1.5
    elif state == TerminalState.FINAL_STATE:
        return 0
    elif state == TerminalState.BAD_OP:
        return -10
    else:
        raise ValueError(f"Unrecognized State: {state}")


def static_rewards(state):
    """
    Returns a reward vector for the state for all actions
    """
    return np.ones(len(actions)) * _reward_amount(state)


def split_ndx_for(state, split_on, dealer_card):
    """
    Returns the state index for the state given:
        - outcome
        - dealer card
        - card being split on
    """
    return len(terminal_states) + (
        (split_on - 1) +  # -1 from these b/c ndx starts at 0
        (dealer_card - 1) * NUM_CARDS +
        state.value * NUM_CARDS * NUM_CARDS
    )


def split_transition_vector(split_on, constructor_args):
    """
    Helper method that returns a transition vector for when
    a split occurs
    """
    indexes = []
    probs = []

    for card, prob in NEXT_CARD_PROBS.items():
        if split_on == 1 or card == 1:
            # Also account for soft Ace if the card is it
            add_to_hand = 10
            more_args = {'is_soft': 1}
        else:
            add_to_hand = 0
            more_args = {}

        if card == split_on:
            state_ndx = State(
                card + split_on + add_to_hand,
                **{**constructor_args, 'can_split': 1, **more_args}
            ).get_state_ndx()
        
        # Blackjack
        elif (split_on == 10 and card == 1) or (split_on == 1 and card == 10):
            # Last split hand, none after this
            if constructor_args['split_on'] == 0:
                state_ndx = TerminalState.BJ.value
            else:
                state_ndx = split_ndx_for(
                    SplitStateEnum.SBJ,
                    split_on,
                    constructor_args['dealer_card']
                )

        else:
            state_ndx = State(
                card + split_on + add_to_hand,
                **{**constructor_args, **more_args}
            ).get_state_ndx()

        indexes.append(int(state_ndx))
        probs.append(prob)

    vector = np.zeros(states_len)
    vector[indexes] = np.array(probs)

    return vector



In [6]:
class State:
    
    def __init__(
            self,
            hand_value,
            dealer_card,
            is_soft,
            can_double,
            can_split,
            split_on
    ):
        """
        :param hand_value: Value of player hand
        :param dealer_card: Value of dealer card
        :param is_soft: If the hand is soft (has an Ace acting as an 11)
        :param can_double: If the hand can double (only if initial deal)
        :param can_split: If the hand can split (only if initial deal and matching cards)
        :param split_on: If the hand was split on a number initially
                         (takes us back to initial deal state after)
        """
        self.hand_value = hand_value
        self.dealer_card = dealer_card
        self.is_soft = is_soft
        self.can_double = can_double
        self.can_split = can_split
        self.split_on = split_on
        
        # Since hand values and what not aren't necessarily indexes,
        # need this to find the corresponding state_ndx
        self.state_val_ndxs = [
            non_terminal_states[ndx].index(val)
            for ndx, val in enumerate([
                hand_value, dealer_card,
                is_soft, can_double, can_split,
                split_on
            ])
        ]
    
    def get_state_ndx(self):
        """
        Returns non-terminal state ndx based on the params provided
        """
        state_ndx = len(static_states)
        
        for ndx, val in enumerate(self.state_val_ndxs):
            curr_ndx_val = val
            
            for i in range(ndx):
                curr_ndx_val *= len(non_terminal_states[i])
            
            state_ndx += curr_ndx_val
        
        return state_ndx
    
    def _hand_comp_outcome_prob(self):
        """
        Gets the distribution of probabilities given this hand
        and the dealer's showing card (win, lose, push)
        :return: tuple with win prob, lose prob, push prob
        """
        return tuple([
            sum([
                DEALER_SCORE_MAP[self.dealer_card][val]
                for val in DEALER_SCORE_MAP[self.dealer_card]
                if comp(val, self.hand_value)
            ])
            for comp in [
                lambda x, y: x < y,
                lambda x, y: x > y,
                lambda x, y: x == y
            ]
        ])
    
    def _hit_hand_outcome_states(self):
        """
        Helper function to get the potential outcome states
        resulting from a HIT action
        :return: a tuple of (index array, probability array)
                 this tuple should be used to assign the transition vector
        """
        if self.is_soft:
            bust_prob = 0
        else:
            bust_prob = sum([
                prob for card, prob in NEXT_CARD_PROBS.items()
                if card + self.hand_value > 21
            ])
        
        same_states = [
            (self.hand_value + card, prob)
            for card, prob in NEXT_CARD_PROBS.items()
            if self.hand_value + card <= 21 and (self.is_soft or card != 1)
        ]
        
        this_is_true = False
        
        # Finding if we are going to transition to a soft state
        soft_states = []
        if not self.is_soft and self.hand_value < 21:
            this_is_true = True
            if self.hand_value < 11:
                soft_states.append((self.hand_value + 11, NEXT_CARD_PROBS[1]))
            else:
                # Actually no change, we still add an ace
                same_states.append((self.hand_value + 1, NEXT_CARD_PROBS[1]))
        
        # Finding if we are going to transition to a hard state
        hard_states = []
        if self.is_soft:
            hard_states += [
                (self.hand_value - 10 + card, prob)
                for card, prob in NEXT_CARD_PROBS.items()
                if self.hand_value + card > 21
            ]
        
        index_list = (
            [TerminalState.Lost.value] +
            [
                self._new_hand_state(new_val).get_state_ndx()
                for new_val, _ in same_states
            ] +
            [
                self._new_hand_state(new_val, is_soft=True).get_state_ndx()
                for new_val, _ in soft_states
            ] +
            [
                self._new_hand_state(new_val, is_soft=False).get_state_ndx()
                for new_val, _ in hard_states
            ]
        )
        
        prob_vector = np.concatenate([
            [bust_prob],
            [prob for _, prob in same_states],
            [prob for _, prob in soft_states],
            [prob for _, prob in hard_states]
        ])
        
        if abs(sum(prob_vector) - 1) > 0.01:
            print(self)
            print(same_states)
            print(soft_states)
            print(hard_states)
            print(bust_prob)
            print(this_is_true)
            print('-' * 80)
        
        return index_list, prob_vector
    
    def get_transition_vector(self, action):
        """
        Returns a (states_len, 1) array containing probabilities of going
        to the corresponding state given the action
        """
        vector = np.zeros(states_len)
        
        # Invalid state, so just cut it off from everything
        # (can't get here, so don't let it go anywhere either)
        if self.invalid_state():
            vector[self.get_state_ndx()] = 1
            return vector
        
        # Invalid operation from the current state, so give it a really
        # bad reward then terminate to discourage
        if self.invalid_action(action):
            vector[TerminalState.BAD_OP.value] = 1
            return vector
        
        if action == PlayerAction.STAY or action == PlayerAction.DOUBLE:
            # Evaluate against dealer hand
            probs = self._hand_comp_outcome_prob()
            
            if self.split_on:
                # Need to do another hand
                # Need multiple starting states for the reward to be accurate I think
                # TODO: Make the reward function reflect what happened
                if action == PlayerAction.STAY:
                    states = [SplitStateEnum.SWon, SplitStateEnum.SLost, SplitStateEnum.SPushed]
                elif action == PlayerAction.DOUBLE:
                    states = [SplitStateEnum.SWon2, SplitStateEnum.SLost2, SplitStateEnum.SPushed]
                
                state_ndxs = [
                    split_ndx_for(state, self.split_on, self.dealer_card)
                    for state in states
                ]
            
            elif action == PlayerAction.STAY:
                state_ndxs = [
                    TerminalState.Won.value,
                    TerminalState.Lost.value,
                    TerminalState.Pushed.value
                ]
            
            elif action == PlayerAction.DOUBLE:
                state_ndxs = [
                    TerminalState.Won2.value,
                    TerminalState.Lost2.value,
                    TerminalState.Pushed.value
                ]

            vector[state_ndxs] = probs
        
        elif action == PlayerAction.HIT:
            indexes, probs = self._hit_hand_outcome_states()
            vector[indexes] = probs
        
        elif action == PlayerAction.SPLIT:
            constructor_args = {
                'dealer_card': self.dealer_card,
                'is_soft': 0,
                'can_double': 1,
                'can_split': 0,
                'split_on': self.hand_value / 2
            }
            return split_transition_vector(self.hand_value / 2, constructor_args)
        
        return vector
    
    def get_reward(self):
        """
        Returns the reward value given the action for the current state
        """
        return np.zeros(len(actions))
    
    def _new_hand_state(self, hand_value, **kwargs):
        """
        Helper function to return a new state the same as this one,
        except with a new hand and maybe some other arguments
        """
        init_params = {
            'hand_value': hand_value,
            'dealer_card': self.dealer_card,
            'is_soft': self.is_soft,
            'can_double': self.can_double,
            'can_split': self.can_split,
            'split_on': self.split_on
        }
        for keyword in kwargs:
            if keyword in init_params:
                init_params[keyword] = kwargs[keyword]
        
        return State(**init_params)
    
    def invalid_state(self):
        """
        Returns if this is an invalid state
        (i.e. is_soft is true but hand_value = 4)
        """
        if self.is_soft and self.hand_value < 12:
            return True
        
        if self.can_split and self.hand_value % 2 != 0:
            return True
    
    def invalid_action(self, action):
        """
        Returns if the given action is invalid from the current state
        (i.e. SPLIT from not having `can_split`)
        """
        if action == PlayerAction.DOUBLE and not self.can_double:
            return True
        
        if action == PlayerAction.SPLIT and not self.can_split:
            return True
    
    def __repr__(self):
        hand_str = self.hand_value
        dealer_str = self.dealer_card
        soft_str = 'Is Soft' if self.is_soft else ''
        double_str = 'Can Double' if self.can_double else ''
        split_str = 'Can Split' if self.can_split else ''
        split_on_str = f"Split on {self.split_on}" if self.split_on else ''
        return f"{hand_str}, {dealer_str}: {','.join([soft_str, double_str, split_str, split_on_str])}"


class SplitIntermediateState:
    """
    This class represents a state in between finishing a hand and continuing
    a split
    """
    def __init__(self, state_type, split_on, dealer_card):
        """
        :param split_on: The card that was split on originally
        :param dealer_card: The card the dealer is showing
        :param state_type: The value of the enum that the last hand resulted in
        """
        self.state_type = state_type
        self.split_on = split_on
        self.dealer_card = dealer_card
    
    def get_state_ndx(self):
        """
        Returns the state index
        """
        return split_ndx_for(self.state_type, self.split_on, self.dealer_card)
    
    def get_reward(self):
        return static_rewards(self.state_type)
    
    def get_transition_vector(self, action):
        constructor_args = {
            'dealer_card': self.dealer_card,
            'is_soft': 0,
            'can_double': 1,
            'can_split': 0,
            'split_on': 0
        }
        return split_transition_vector(self.split_on, constructor_args)


In [26]:
# Defining transitions and rewards

transitions = [
    np.zeros((states_len, states_len))
    for _ in actions
]
rewards = np.zeros((states_len, len(actions)))

states_considered = set()  # Going to add to this set to make sure we cover all states
state_ndx_map = {}  # Need a way to map each state back to human readable form

total_start_time = time.time()

for tstate in terminal_states:
    for action in actions:
        transitions[action.value][tstate.value, :] = terminal_transitions(tstate)
        rewards[tstate.value] = static_rewards(tstate)
        
        states_considered.add(tstate.value)

print(states_considered)

for split_num in range(1, NUM_CARDS + 1):
    for dealer_card in range(1, NUM_CARDS + 1):
        for istate in split_intermediate_states[:NUM_INTERMEDIATE_STATES]:
            
            state = SplitIntermediateState(
                istate,
                split_num,
                dealer_card
            )
            state_ndx = state.get_state_ndx()
            
            for action in actions:
                transitions[action.value][state_ndx, :] = state.get_transition_vector(action)
            rewards[state_ndx] = state.get_reward()
            
            states_considered.add(state_ndx)

print(len(states_considered))

for hand_val in hand_states:
    for dealer_card in dealer_card_states:
        print(f"Hand: {hand_val}, Dealer: {dealer_card} - ", end='')
        tic()

        for is_soft in is_soft_states:
            for can_double in can_double_states:
                for can_split in can_split_states:
                    for split_on in split_on_states:

                        state = State(
                            hand_val, dealer_card, is_soft, can_double, can_split, split_on
                        )
                        state_ndx = state.get_state_ndx()

                        for action in actions:
                            transitions[action.value][state_ndx, :] = state.get_transition_vector(action)
                        rewards[state_ndx] = state.get_reward()
                        
                        states_considered.add(state_ndx)
                        state_ndx_map[state_ndx] = state

        toc()

#transitions = [
#    csr_matrix(transition) for transition in transitions
#]
#rewards = csr_matrix(rewards)

print()
print(f"Total Time: {time.time() - total_start_time}")

{0, 1, 2, 3, 4, 5, 6, 7}
608
Hand: 4, Dealer: 1 - 0.0525670051574707
Hand: 4, Dealer: 2 - 0.05588102340698242
Hand: 4, Dealer: 3 - 0.05842995643615723
Hand: 4, Dealer: 4 - 0.09333610534667969
Hand: 4, Dealer: 5 - 0.05134987831115723
Hand: 4, Dealer: 6 - 0.050601959228515625
Hand: 4, Dealer: 7 - 0.05200982093811035
Hand: 4, Dealer: 8 - 0.07868075370788574
Hand: 4, Dealer: 9 - 0.0578610897064209
Hand: 4, Dealer: 10 - 0.047493934631347656
Hand: 5, Dealer: 1 - 0.0387272834777832
Hand: 5, Dealer: 2 - 0.04053378105163574
Hand: 5, Dealer: 3 - 0.04671502113342285
Hand: 5, Dealer: 4 - 0.04115605354309082
Hand: 5, Dealer: 5 - 0.03853917121887207
Hand: 5, Dealer: 6 - 0.08972024917602539
Hand: 5, Dealer: 7 - 0.09580397605895996
Hand: 5, Dealer: 8 - 0.04883718490600586
Hand: 5, Dealer: 9 - 0.045324087142944336
Hand: 5, Dealer: 10 - 0.037325143814086914
Hand: 6, Dealer: 1 - 0.042346954345703125
Hand: 6, Dealer: 2 - 0.054818153381347656
Hand: 6, Dealer: 3 - 0.04253101348876953
Hand: 6, Dealer: 4 - 0.

In [8]:
len(states_considered)

16448

In [32]:
for i in range(states_len):
    if i not in states_considered:
        print(i)

## MDP TIME

In [10]:
fh = mdptoolbox.mdp.FiniteHorizon(
    transitions,
    rewards,
    0.99,
    20
)
fh.setVerbose()
fh.run()

stage: 19, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

stage: 18, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

stage: 17, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

stage: 16, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

stage: 15, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

stage: 4, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

stage: 3, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

stage: 2, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

stage: 1, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

stage: 0, policy: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [27]:
pi = mdptoolbox.mdp.PolicyIterationModified(
    transitions,
    rewards,
    0.99,
    epsilon=0.0000000001
)
pi.setVerbose()
pi.run()

  	Iteration		V-variation
    1		  12.0
    2		  12.089855625222867
    3		  3.069057420695117
    4		  0.9012757168173948
    5		  0.2523826878231148
    6		  6.204867872838804e-06
    7		  1.0288090379617643e-09
    8		  5.684341886080801e-13


In [35]:
sum(list(pi.policy)[:608])

0

## Converting the Policy Back to Readability

In [30]:
def convert_state_to_dict(state):
    return {
        'hand_value': state.hand_value,
        'dealer_card': state.dealer_card,
        'is_soft': state.is_soft,
        'can_split': state.can_split,
        'can_double': state.can_double,
        'split_on': state.split_on
    }

for state_ndx in state_ndx_map:
    state_ndx_map[state_ndx] = convert_state_to_dict(state_ndx_map[state_ndx])

In [31]:
state_ndx_map

{608: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 0},
 2048: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 1},
 3488: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 2},
 4928: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 3},
 6368: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 4},
 7808: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 5},
 9248: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 6},
 10688: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_split': 0,
  'can_double': 0,
  'split_on': 7},
 12128: {'hand_value': 4,
  'dealer_card': 1,
  'is_soft': 0,
  'can_spl

In [36]:
with open('data/tmp/basic_hand_strategy_mdp_results.json', 'w') as tmp_file:
    json.dump(
        {
            'state_ndx_map': state_ndx_map,
            'action_ndx_map': {
                0: 'STAY',
                1: 'HIT',
                2: 'SPLIT',
                3: 'DOUBLE'
            },
            'policy': list(pi.policy)
        },
        tmp_file
    )

## Playing the Game with our Agent

In [7]:
from src.simulator import BaseGame, Player, Deck
from src.agents.dealers import SimpleDealer
from src.agents.betting import ConstantBettingAgent
from src.simulator.player import PlayerHandAgent

In [8]:
with open('data/tmp/basic_hand_strategy_mdp_results.json') as tmp_file:
    policy_info = json.load(tmp_file)

In [13]:
class BasicMDPAgent(PlayerHandAgent):
    def __init__(self):
        super(BasicMDPAgent, self).__init__()
        
        self.hand_value = None
        self.dealer_card = None
        self.is_soft = None
        self.can_split = None
        self.can_double = None
        self.split_on = None
    
    def update_model(self, game):
        hand = self.hands[self.curr_hand_ndx]
        
        self.hand_value = hand.value()
        self.dealer_card = game.dealer.hands[0].cards[0]
        self.is_soft = int(hand.is_soft())
        self.can_split = int(len(hand.cards) == 2)
        self.can_double = int(len(hand.cards) == 2)
        
        if len(self.hands) == self.curr_hand_ndx - 1:
            self.split_on = 0
        else:
            self.split_on = self.hands[self.curr_hand_ndx + 1].cards[0]
    
    def get_action(self):
        state = State(
            self.hand_value,
            self.dealer_card,
            self.is_soft,
            self.can_split,
            self.can_double,
            self.split_on
        )
        action_ndx = policy_info['policy'][state.get_state_ndx()]
        
        if action_ndx == 0:
            return PlayerAction.STAY
        elif action_ndx == 1:
            return PlayerAction.HIT
        elif action_ndx == 2:
            return PlayerAction.DOUBLE
        elif action_ndx == 3:
            return PlayerAction.SPLIT


In [14]:
game = BaseGame(
    Deck(),
    SimpleDealer(),
    Player(
        BasicMDPAgent(),
        ConstantBettingAgent()
    ),
    debug=True
)

In [15]:
game.play_chute()

Player Cash: 50, Current Hand:  bets 10
Dealer showing 1
Results:


IndexError: list index out of range